In [1]:
import os, time, shutil, tarfile
from minio import Minio
from tqdm import tnrange

times = 10
resultSmallFile = []
resultBigFile = []
testList = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Gen test file

In [2]:
with open('file.txt', 'w') as f:
    for i in range(64*1000*1000):
        f.write('a')
        
try:
    shutil.rmtree('minio_test_data')
except:
    pass

os.mkdir('minio_test_data')

for size in testList:
    os.mkdir('minio_test_data/'+str(size))
    tar=tarfile.open('minio_test_data/'+str(size)+'.tar','w')  
    for i in range(size):
        shutil.copyfile('file.txt', 'minio_test_data/'+str(size)+'/'+str(i))
        tar.add('minio_test_data/'+str(size)+'/'+str(i))
    tar.close()

Upload test

In [2]:
minioClient = Minio('140.124.181.78:31312',
                    access_key='wei840222',
                    secret_key='j04rmp4jo3',
                    secure=False)

for ti in tnrange(times, desc='times'):
    runTimeListSmallFile = []
    runTimeListBigFile = []
    for size in testList:
        minioClient.make_bucket('test-size'+str(size)+'-'+str(ti), location="us-east-1")
        with tnrange(size, desc='files') as t:
            try:
                start = time.time()
                for j in t:
                    minioClient.fput_object('test-size'+str(size)+'-'+str(ti), str(j), 'minio_test_data/'+str(size)+'/'+str(j))
                end = time.time()
                runTimeListSmallFile.append(end-start)
            except:
                print('upload minio_test_data/'+str(size)+'/'+str(j)+'fail')
                runTimeListSmallFile.append(-1)
            try:
                start = time.time()
                minioClient.fput_object('test-size'+str(size)+'-'+str(ti), str(size)+'.tar', 'minio_test_data/'+str(size)+'.tar')
                end = time.time()
                runTimeListBigFile.append(end-start)
            except:
                print('upload minio_test_data/'+str(size)+'.tar fail')
                runTimeListBigFile.append(-1)
    resultSmallFile.append(runTimeListSmallFile)
    resultBigFile.append(runTimeListBigFile)

output result to csv

In [6]:
with open('resultSmallFile.csv', 'w') as f:
    f.write(str([str(t)+'-file' for t in testList]).replace("'",'')[1:-1]+'\n')
    for r in resultSmallFile:
        f.write(str(r)[1:-1]+'\n')

In [7]:
with open('resultBigFile.csv', 'w') as f:
    f.write(str([str(t)+'-file' for t in testList]).replace("'",'')[1:-1]+'\n')
    for r in resultBigFile:
        f.write(str(r)[1:-1]+'\n')